In [12]:
from mlsocket import MLSocket
import time
import numpy as np
import cv2

milli_time = lambda: int(round(time.time() * 1000))

HOST = '192.168.0.1'
# HOST = '192.168.1.230'
PORT = 7777

socket = MLSocket()                             # КЛИЕНТ
socket.connect((HOST, PORT)) 
print('Socket connected')

video_ind = 195
print('video_ind =', video_ind)

milli_time = lambda: int(round(time.time() * 1000))
video_delay = milli_time()
frame = np.zeros([120, 72, 3])
video = np.array([frame])

while True:
    if milli_time() - video_delay > 5000:
        video = socket.recv(4096)
        print(video_ind, ':', video.shape)
        np.save(f'Videos pass obstacle/obstacle_left/video{video_ind}', video)
        video_delay = milli_time()
        video_ind += 1
        for frame in video:
            frame = cv2.resize(frame, (frame.shape[1]*1, frame.shape[0]*1))
            cv2.imshow(' ', frame)
            if cv2.waitKey(55) & 0xFF == ord('q'):
                socket.close()
                cv2.destroyAllWindows()
                break
    else:
        BREAK = False
        for frame in video:
            frame = cv2.resize(frame, (frame.shape[1]*1, frame.shape[0]*1))
            cv2.imshow(' ', frame)
            if cv2.waitKey(55) & 0xFF == ord('q'):
                socket.close()
                cv2.destroyAllWindows()
                BREAK = True
                break
        if BREAK == True:
            break



Socket connected
video_ind = 195
195 : (24, 288, 480, 3)
196 : (24, 288, 480, 3)
197 : (24, 288, 480, 3)
198 : (24, 288, 480, 3)
199 : (24, 288, 480, 3)
200 : (24, 288, 480, 3)
201 : (24, 288, 480, 3)
202 : (24, 288, 480, 3)
203 : (24, 288, 480, 3)
204 : (24, 288, 480, 3)
205 : (24, 288, 480, 3)
206 : (24, 288, 480, 3)
207 : (24, 288, 480, 3)
208 : (24, 288, 480, 3)
209 : (24, 288, 480, 3)
210 : (24, 288, 480, 3)
211 : (24, 288, 480, 3)
212 : (24, 288, 480, 3)
213 : (24, 288, 480, 3)
214 : (24, 288, 480, 3)
215 : (24, 288, 480, 3)


In [5]:
import numpy as np
import cv2
import time

video_ind = 186
while True:
    try:
        video = np.load('Videos pass obstacle/obstacle_right/video%d.npy' % video_ind)
    except:
        break

    print(video_ind, len(video))
    BREAK = False
    for frame in video:
#         frame = cv2.adaptiveThreshold(frame, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 15, 12)
        frame = cv2.resize(frame, (frame.shape[1]*4, frame.shape[0]*4))
        cv2.imshow('1', frame)
        if cv2.waitKey(55) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            BREAK = True
            break
    if BREAK == True:
        break
    
#     time.sleep(2)
    
    video_ind += 1
    
cv2.destroyAllWindows()

In [4]:
video.shape

NameError: name 'video' is not defined

In [14]:

import numpy as np
import cv2
import time

milli_time = lambda: int(round(time.time() * 1000))
    
cap = cv2.VideoCapture(2)

ret, frame = cap.read()
frame = cv2.resize(frame, (240, 90))
t = milli_time()
t_frame = milli_time()
t_delay_between_videos = milli_time()

video = []
video_ind = 0
delay_video = True

while True:
    if delay_video == True and milli_time() - t_delay_between_videos > 4000:
        delay_video = False
        t = milli_time()
        
    if milli_time() - t_frame > 100:
        t_frame = milli_time()
        
        ret, frame = cap.read()
        frame = cv2.resize(frame, (240, 90))
        frame = frame[0:, 0:int(frame.shape[1]/2)]
        frame = cv2.bilateralFilter(frame, 7, 75, 75)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        if delay_video == False:
            Forward()

            frame = np.array(frame)
            video.append(frame)

            if milli_time() - t > 1500:
                print(video_ind, ':', len(video))
                np.save('videos/video%d' % video_ind, video)
                video.clear()
                video_ind += 1
                t_delay_between_videos = milli_time()
                delay_video = True
                ExtraStop()

    cv2.imshow('video feed', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [1]:
import socket
import sys
import cv2
import pickle
import numpy as np
import struct
from PIL import Image
import keyboard
import time

milli_time = lambda: int(round(time.time() * 1000))
 
PORT = 7777
 
s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print('Socket created')
 
s.bind(('192.168.207.214', PORT))
print('Socket bind complete')
s.listen(1)
print('Socket now listening')

video_ind = 167
print('video_ind =', video_ind)
 
conn, addr = s.accept()

print('Addr', conn, addr)
 
data = b''
payload_size = struct.calcsize("L")

t = milli_time()
t_delay_between_videos = milli_time()
last = milli_time()

video = []
delay_video = True

while True:
    while len(data) < payload_size:
        data += conn.recv(4096)

    if data.isupper():
        if 'BYE' in data.decode():
            break

    packed_msg_size = data[:payload_size]

    data = data[payload_size:]
    msg_size = struct.unpack("L", packed_msg_size)[0]

    while len(data) < msg_size:
        data += conn.recv(4096)
    frame_data = data[:msg_size]
    data = data[msg_size:]

    frame=pickle.loads(frame_data)
        
    if delay_video == False:
        frame = np.array(frame)
        video.append(frame)

        if milli_time() - t > 1500:
            print(video_ind, ':', len(video))
            np.save('videos/stop/video%d' % video_ind, video)
            video.clear()
            video_ind += 1
            t_delay_between_videos = milli_time()
            delay_video = True
        
    if delay_video == True and milli_time() - t_delay_between_videos > 8000:
        delay_video = False
        t = milli_time()
        
    if delay_video == False:
        conn.send(str.encode('forward'))
#         conn.send(str.encode('extra stop'))
    else:
        conn.send(str.encode('extra stop'))

    frame = cv2.resize(frame, (frame.shape[1]*4, frame.shape[0]*4))
    cv2.imshow('Заголовок окна', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# root.destroy()
# s.close()
cv2.destroyAllWindows()
conn.close()
print('BB')

Socket created


OSError: [WinError 10049] Требуемый адрес для своего контекста неверен

In [5]:
 
    
    cv2.destroyAllWindows()
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    